In [20]:
import matplotlib.pyplot as plt
import pickle
import os
from itertools import combinations
import numpy as np
import collections
from IPython.lib import backgroundjobs as bg
from tqdm import tqdm_notebook
import heapq
from gensim.models import Word2Vec
import node2vec
import networkx as nx
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [2]:
with open('./data/2016_q1_cci-neighbours_kcore.pickle', 'rb') as picklefile:
    neighbours_q1 = pickle.load(picklefile)

In [3]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)

In [4]:
dimensions=32
window=10
iterations=10
p=1
q=1
num_walks=25
walk_length = 400

In [5]:
nx_weighted = nx.Graph()
nx_weighted.add_nodes_from(neighbours_q1.keys())
for edge in weights_q1:
    u, v = edge[0]
    nx_weighted.add_edge(u, v, weight=edge[1])

In [7]:
G = node2vec.Graph(nx_weighted, False, p, q)
%time G.preprocess_transition_probs()

CPU times: user 34min 12s, sys: 13.7 s, total: 34min 25s
Wall time: 34min 29s


In [8]:
%time walks = G.simulate_walks(num_walks, walk_length)
walks = [map(str, walk) for walk in walks]
%time model = Word2Vec(walks, size=dimensions, window=window, min_count=0, sg=1, iter=iterations)

Walk iteration:
1 / 25
2 / 25
3 / 25
4 / 25
5 / 25
6 / 25
7 / 25
8 / 25
9 / 25
10 / 25
11 / 25
12 / 25
13 / 25
14 / 25
15 / 25
16 / 25
17 / 25
18 / 25
19 / 25
20 / 25
21 / 25
22 / 25
23 / 25
24 / 25
25 / 25
CPU times: user 1h 7min 4s, sys: 1min 35s, total: 1h 8min 39s
Wall time: 1h 11min 40s
CPU times: user 29min 20s, sys: 4.58 s, total: 29min 25s
Wall time: 9min 51s


In [32]:
# with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
#     weights_q1 = pickle.load(picklefile)
# with open('./data/2016_q2_cci-weights_kcore.pickle', 'rb') as picklefile:
#     weights_q2 = pickle.load(picklefile)
edges_q1 = set(edge for edge, weight in weights_q1)
edges_q1 = set((u, v) if u < v else (v, u) for u, v in edges_q1)
edges_q2 = set(edge for edge, weight in weights_q2)
edges_q2 = set((u, v) if u < v else (v, u) for u, v in edges_q2)
new_edges=edges_q2.difference(edges_q1)
k = len(new_edges)
k

26971

In [ ]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

100, 25 (2864, 0.11310323039254404)
100, 100 (3459, 0.1282488598865448)

In [19]:
len(neighbours_q1.keys())

3053

In [10]:
authors = neighbours_q1.keys()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

/Users/yilingc/Courses/CS224w/project/GithubNetworkAnalysis/snap-2.7/env2.7/lib/python2.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


(3522, 0.13058470208742723)

In [ ]:
len(project_authors_q1['author_id'].unique())

In [ ]:
pairs = list(combinations(authors, 2))
alldist = {}
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    alldist[(u, v)] = model.wv.distance(str(u), str(v))
with open('./data/node2vec-weighted-p10-q01-100-100.pickle', 'wb') as picklefile:
    pickle.dump(alldist, picklefile)

In [33]:
authors = neighbours_q1.keys()
pairs = [(u, v) if u < v else (v, u) for u, v in combinations(authors, 2)]
truth = [edge in edges_q2 for edge in pairs]
scores = [model.wv.similarity(str(u), str(v)) for u, v in pairs]
fpr, tpr, thres = metrics.roc_curve(truth,scores)
metrics.auc(fpr,tpr), metrics.average_precision_score(truth,scores)

(0.9434675141568222, 0.35512693643116466)

In [38]:
edgerep = [np.multiply(model.wv.get_vector(str(u)), model.wv.get_vector(str(v))) for u, v in pairs]

In [19]:
edgerep[0]

array([ 1.99326756e-03,  3.55617665e-02, -1.03616752e-01,  2.19479442e-01,
        7.47208968e-02, -2.34495580e-01,  2.60082692e-01,  1.53253555e-01,
        2.84805503e-02,  2.01972112e-01,  4.59548645e-02,  9.46486145e-02,
       -2.53715869e-02, -5.73649034e-02,  1.06381172e-04,  7.26601575e-03,
        4.19349484e-02,  7.00017884e-02, -1.85569432e-02, -1.19985454e-01,
       -3.21399756e-02,  2.43899494e-01, -4.24645171e-02,  2.16103330e-01,
        6.43621624e-01, -3.49727869e-01,  5.09458423e-01,  1.10678084e-01,
        1.80659875e-01,  2.93642312e-01, -4.68817711e-01, -1.26549587e-01],
      dtype=float32)

In [34]:
minus_q2 = list(set(pairs).difference(edges_q2))

In [35]:
y_train = [edge in edges_q1 for edge in minus_q2]

In [36]:
sum(y_train), len(edges_q1.difference(edges_q2))

(338600, 338600)

In [50]:
(model.wv.get_vector(str(u))- model.wv.get_vector(str(v)))**2

array([5.44051565e-02, 9.27825421e-02, 1.17456876e-01, 9.95067954e-01,
       5.95249653e-01, 1.54920375e+00, 3.28084175e-03, 1.75502311e-04,
       1.97557993e-02, 5.48107445e-01, 7.69455126e-03, 4.41962898e-01,
       2.77047098e-01, 2.03030825e+00, 4.77981955e-01, 2.06815839e-01,
       2.25393131e-04, 2.14937814e-02, 1.95710363e-05, 3.06967020e-01,
       1.35606930e-01, 2.07813442e-01, 3.50289792e-01, 2.45498121e-01,
       3.72117907e-01, 3.86738062e-01, 7.13143647e-02, 1.30053088e-02,
       2.58672833e-02, 3.62732522e-02, 6.47538364e-01, 6.02031112e-01],
      dtype=float32)

In [37]:
x_train = [np.multiply(model.wv.get_vector(str(u)), model.wv.get_vector(str(v))) for u, v in minus_q2]

In [46]:
x_train = [abs(model.wv.get_vector(str(u))- model.wv.get_vector(str(v))) for u, v in minus_q2]

In [47]:
edgerep = [abs(model.wv.get_vector(str(u))- model.wv.get_vector(str(v))) for u, v in pairs]

In [51]:
x_train = [(model.wv.get_vector(str(u))- model.wv.get_vector(str(v)))**2 for u, v in minus_q2]
edgerep = [(model.wv.get_vector(str(u))- model.wv.get_vector(str(v)))**2 for u, v in pairs]

In [48]:
lg = LogisticRegression(penalty='l2',C=0.001)
lg.fit(x_train,y_train)
lg_y_pred_est = lg.predict_proba(edgerep)[:,1]
fpr,tpr,thresholds = metrics.roc_curve(truth,lg_y_pred_est)
metrics.auc(fpr,tpr), metrics.average_precision_score(truth,lg_y_pred_est)

(0.9496093289258162, 0.38039147517132016)

(0.935332837824568, 0.31882295272929934)

In [52]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(penalty='l2',C=0.001)
lg.fit(x_train,y_train)
lg_y_pred_est = lg.predict_proba(edgerep)[:,1]
fpr,tpr,thresholds = metrics.roc_curve(truth,lg_y_pred_est)
metrics.auc(fpr,tpr), metrics.average_precision_score(truth,lg_y_pred_est)

(0.9495045317398229, 0.3810955830785046)